# Clean up the full train and test datasets

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pprint
from IPython.display import display, HTML
from sklearn.preprocessing import Imputer


## Load Data

In [4]:
train_file = "../data/train.csv"
train = pd.read_csv(train_file, index_col=0)

test_file = "../data/test.csv"
test = pd.read_csv(test_file, index_col=0)

## Inspect data
Lets check to see if train and test data are ordered the same way... 

In [89]:
names = [] 
different_names = []
for i in range(len(test.columns)):
    names.append((i, train.columns[i], test.columns[i]))
    if not train.columns[i] == test.columns[i]:
        print("{}:{} name different!".format(i,train.columns[i]))
        different_names.append((i, train.columns[i], test.columns[i]))

print(len(different_names))

0


And if read_csv has applied the same data type to both train and test data

In [90]:
dtypes = []
different_dtypes = []
for i in range(len(test.columns)):
    dtypes.append((i, train.dtypes[i], test.dtypes[i]))
    if not train.dtypes[i] == test.dtypes[i]:
        print("{}:{} dtypes different!".format(i,train.columns[i]))
        different_dtypes.append((i, train.dtypes[i], test.dtypes[i]))
print(len(different_dtypes))

33:BsmtFinSF1 dtypes different!
35:BsmtFinSF2 dtypes different!
36:BsmtUnfSF dtypes different!
37:TotalBsmtSF dtypes different!
46:BsmtFullBath dtypes different!
47:BsmtHalfBath dtypes different!
60:GarageCars dtypes different!
61:GarageArea dtypes different!
8


In [91]:
different_dtypes

[(33, dtype('int64'), dtype('float64')),
 (35, dtype('int64'), dtype('float64')),
 (36, dtype('int64'), dtype('float64')),
 (37, dtype('int64'), dtype('float64')),
 (46, dtype('int64'), dtype('float64')),
 (47, dtype('int64'), dtype('float64')),
 (60, dtype('int64'), dtype('float64')),
 (61, dtype('int64'), dtype('float64'))]

Inspecting those differences, we see that they're all float vs int (no categorical object vs number).  Wonder why it interpreted some as int and some as float... Does the test data have fractions?

In [116]:
# Make a copy to do this, so I can add in two control columns to make sure my syntax is correct...
find_remainders = test.iloc[:, [x[0] for x in different_dtypes]].copy()
find_remainders['float'] = 1.5
find_remainders['int'] = 1

print(find_remainders.dtypes)
print((find_remainders % 1 > 0).any())

BsmtFinSF1      float64
BsmtFinSF2      float64
BsmtUnfSF       float64
TotalBsmtSF     float64
BsmtFullBath    float64
BsmtHalfBath    float64
GarageCars      float64
GarageArea      float64
float           float64
int               int64
dtype: object
BsmtFinSF1      False
BsmtFinSF2      False
BsmtUnfSF       False
TotalBsmtSF     False
BsmtFullBath    False
BsmtHalfBath    False
GarageCars      False
GarageArea      False
float            True
int             False
dtype: bool


No fractions in test data.  Are there nan?

In [118]:
pd.isnull(train.iloc[:, [x[0] for x in different_dtypes]]).any()

BsmtFinSF1      False
BsmtFinSF2      False
BsmtUnfSF       False
TotalBsmtSF     False
BsmtFullBath    False
BsmtHalfBath    False
GarageCars      False
GarageArea      False
dtype: bool

In [119]:
pd.isnull(test.iloc[:, [x[0] for x in different_dtypes]]).any()

BsmtFinSF1      True
BsmtFinSF2      True
BsmtUnfSF       True
TotalBsmtSF     True
BsmtFullBath    True
BsmtHalfBath    True
GarageCars      True
GarageArea      True
dtype: bool

Aha!  Seems like presence of NAN might have been what forced them to be float.  I think np.int64 cannot have NAN?  Yeah, see this: http://stackoverflow.com/questions/11548005/numpy-or-pandas-keeping-array-type-as-integer-while-having-a-nan-value

In [120]:
nan_rows = pd.isnull(test.iloc[:, [x[0] for x in different_dtypes]]).any(1).nonzero()[0]

In [235]:
test.iloc[nan_rows, [x[0] for x in different_dtypes]]

,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageCars,GarageArea
Id,,,,,,,,
2121,NaN,NaN,NaN,NaN,NaN,NaN,1.0,280.0
2189,0.0,0.0,0.0,0.0,NaN,NaN,2.0,624.0
2577,548.0,0.0,311.0,859.0,0.0,0.0,NaN,NaN


What about our other floats...?  Are they also floats because of this? 

In [217]:
# Find all other floats (use train because we know the extra floats above from test are not in train)
cols_float = train.select_dtypes(include=[np.float64]).columns.tolist()
# dtypes = {'int':np.int64,
#           'float':np.float64,
#           'other':np.object,}
# col_by_dt = {}
# for dt in dtypes.keys():
#     col_by_dt[dt] = train.columns[train.dtypes == dtypes[dt]].tolist()

In [218]:
print("Columns cast as float: {}".format(cols_float))

Columns cast as float: ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']


Do any of these floats have non-integer numbers?

In [219]:
(train[cols_float] % 1 ).any()

LotFrontage    False
MasVnrArea     False
GarageYrBlt    False
dtype: bool

Do they have NaNs?

In [221]:
print(pd.isnull(train[cols_float]).any())
print("Rows with NaNs:")
print(pd.isnull(train[cols_float]).any(1).nonzero()[0])

LotFrontage    True
MasVnrArea     True
GarageYrBlt    True
dtype: bool
Rows with NaNs:
[   7   12   14   16   24   31   39   42   43   48   50   64   66   76   78
   84   88   89   95   99  100  104  108  111  113  116  120  125  126  127
  131  133  136  140  147  148  149  152  153  155  160  163  165  166  169
  170  177  180  186  191  198  203  207  208  210  214  218  221  234  237
  241  244  249  250  269  287  288  291  293  307  308  310  319  328  330
  335  342  346  347  351  356  360  361  364  366  369  370  375  384  386
  392  393  404  405  412  421  426  431  434  441  447  452  457  458  459
  464  465  470  484  490  495  496  516  518  520  528  529  533  535  537
  538  539  541  545  559  560  562  564  569  580  582  593  610  611  612
  613  614  616  620  623  626  635  636  638  641  645  649  650  660  666
  668  672  679  682  685  687  690  705  706  709  710  714  720  721  726
  734  738  745  746  750  751  757  770  783  784  785  789  791  794  811


In [222]:
train[pd.isnull(train[cols_float]).any(1)][cols_float]

,LotFrontage,MasVnrArea,GarageYrBlt
Id,,,
8,NaN,240.0,1973.0
13,NaN,0.0,1962.0
15,NaN,212.0,1960.0
17,NaN,180.0,1970.0
25,NaN,0.0,1968.0
32,NaN,0.0,1966.0
40,65.0,0.0,NaN
43,NaN,0.0,1983.0
44,NaN,0.0,1977.0


Definitely...  So really we have either ints (with or without NaN's), and other (currently objects - likely categorical?)

## Process the numeric data with NaN

**Should come back and clean this printing up.  For now, just cut-and-pasted the above visuals that we had used before to check**

Get the numeric columns

In [232]:
cols_numeric = train.select_dtypes(include=[np.number]).columns.tolist()

# Don't want to impute on sale price...
cols_numeric.remove('SalePrice')
print(cols_numeric)

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']


Make an imputer for the numeric data

In [233]:
# Look down each column and replace missing values with the median of that column 
# (since we're using integers, median seemed to make more sense)
imp = Imputer(missing_values='NaN', strategy='median', axis=0)
imp.fit(train[cols_numeric])

Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)

Use imputer to fill in the blanks!

In [234]:
train_imputed = train.copy()
train_imputed[cols_numeric] = imp.transform(train_imputed[cols_numeric])
test_imputed  = test.copy()
test_imputed[cols_numeric]  = imp.transform(test_imputed[cols_numeric])

And looking back at that data we were missing before...

In [230]:
train_imputed[pd.isnull(train[cols_float]).any(1)][cols_float]

,LotFrontage,MasVnrArea,GarageYrBlt
Id,,,
8,69.0,240.0,1973.0
13,69.0,0.0,1962.0
15,69.0,212.0,1960.0
17,69.0,180.0,1970.0
25,69.0,0.0,1968.0
32,69.0,0.0,1966.0
40,65.0,0.0,1980.0
43,69.0,0.0,1983.0
44,69.0,0.0,1977.0


In [236]:
test_imputed.iloc[nan_rows, [x[0] for x in different_dtypes]]

,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageCars,GarageArea
Id,,,,,,,,
2121,383.5,0.0,477.5,991.5,0.0,0.0,1.0,280.0
2189,0.0,0.0,0.0,0.0,0.0,0.0,2.0,624.0
2577,548.0,0.0,311.0,859.0,0.0,0.0,2.0,480.0


# Handle the non-numeric columns

## sklearn has OneHotEncoder, but pandas now has a built in get_dummies()
see:
* http://fastml.com/converting-categorical-data-into-numbers-with-pandas-and-scikit-learn/
* http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html

Let's keep track of these hidden int columns and cast them back to int once we've treated the NaN's...